## Подзапросы и Common Table Expressions

### Описание задачи
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".  

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

**Активным** считается пользователь, за все время решивший **больше 10 задач** правильно *в любых дисциплинах*.

**Активным по математике** считается пользователь, за все время решивший **2 или больше задач** правильно *по математике*.


### Дано: *3 таблицы*

#### 1. Таблица *peas*:

|Название атрибута |	Тип атрибута | Смысловое значение | 
|:---|:---:|:---|
|st_id		|int		|ID ученика|
|timest		|timestamp	|Время решения карточки|
|correct	|bool		|Правильно ли решена горошина?|
|subject	|text		|Дисциплина, в которой находится горошина|


#### 2. Таблица *studs*:

|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|st_id	|int	 |ID ученика|
|test_grp	|text	 |Метка ученика в данном эксперименте|

#### 3. Таблица *final_project_check*:

|Название атрибута	|Тип атрибута	|Смысловое значение
|:---|:---:|:---|
|st_id	|int 	|ID ученика
|sale_time	|timestamp	|Время покупки
|money	|int	|Цена, по которой приобрели данный курс
|subject	|text 	|Дисциплина

### Задача: Необходимо в одном запросе выгрузить следующую информацию о группах пользователей *(pilot и control)*:
* **ARPU** 
* **ARPAU** 
* **CR в покупку** 
* **СR активного пользователя в покупку** 
* **CR пользователя из активности по математике** *(subject = ’Мath’)* **в покупку курса по математике** 

### Решение задачи: 
1. Для того, чтобы вывести требуемую информацию в одном запросе, предлагаю считать метрики на основе Common Table Expression.

2. Определим, какие данные необходимы для расчета требуемых метрик: 

    * **ARPU** - сумма покупок, деленная на число пользователей;
    * **ARPAU** - сумма покупок, деленная  на число **активных** пользователей;
    * **CR в покупку** - число купивших курс юзеров, деленное на общее число юзеров;
    * **СR активного пользователя в покупку** - число купивших курс **активных** юзеров, деленное на общее число **активных** юзеров;
    * **CR пользователя из активности по математике** *(subject = ’Мath’)* **в покупку курса по математике** - число покупок курса по математике, деленное на число активных по математике юзеров.
    
3. Создадим CTE, в которой выведем следующие поля: 

|Название поля	|Тип поля	|Исходная таблица|Смысловое значение | 
|:---|:---:|:---|:---|
|st_id				|int 	|studs 					|ID ученика 					 					|
|test_grp			|text	|studs 					|Принадлежность к группе *(test / control)*			|
|money				|int	|final_project_check	|Выручка за продажу курса *(или Null)*				|
|course_purchase	|int	|final_project_check	|Был ли куплен курс по математике *(Math или Null)*	|
|ttl_solved			|int	|peas					|Число выполненных верно горошин					|
|ttl_solved_math	|int	|peas					|Число выполненных верно горошин по математике		|
  
Этих данных должно быть достаточно, чтобы подсчитать необходимые метрики.

4. Т.к. некоторые студенты могли купить несколько курсов, сгруппируем по ID и группе пользователя. 

5. Для поля money выполним агрегацию SUM, а остальные поля перечислим в GROUP BY, т.к. они идентичны для st_id.

6. На основе CTE посчитаем необходимые метрики с группировкой по *test_grp* и использованием коррелированных подзапросов в SELECT *(кроме ARPU)*. Используем явную *(::float)* или неявную _(*100.00)_ конвертацию в float в промежуточных вычислениях, чтобы получить более точные значения.



```
WITH money_data AS (
                     SELECT s.st_id, 
                            s.test_grp,
                            SUM(f.money) AS money,
                            (SELECT inr_f.subject 
                               FROM final_project_check AS inr_f
                              WHERE inr_f.st_id=s.st_id AND inr_f.subject='Math') AS course_purchase,
                           a.ttl_solved,
                           a.ttl_solved_math 
                       FROM studs AS s                                         
                       LEFT JOIN final_project_check AS f ON s.st_id=f.st_id  --объединим studs с final_project_check
                       LEFT JOIN (                                            --объединим studs с подзапросом из peas 
                                                                              --выведем из таблицы peas:
                                  SELECT ou_p.st_id,                             --ID студента для выполнения JOIN
                                         COUNT(ou_p.correct) AS ttl_solved,      --число верно выполненных горошин 
                                         (SELECT COUNT(inr_p.correct)            --коррелированный подзапрос для числа   
                                            FROM peas as inr_p                     --решенных горошин по математике
--в WHERE внутри подзапроса фильтрация по correct=TRUE не нужна, т.к. из подзапроса будут вытащены только 
  --значения, соответствующие полю ID во внешнем запросе, в котором уже есть такой фильтр
                                           WHERE ou_p.st_id = inr_p.st_id AND subject = 'Math') AS ttl_solved_math
                                    FROM peas as ou_p
                                   WHERE correct=TRUE
                                   GROUP BY st_id
                                  ) AS a ON a.st_id=s.st_id
                      GROUP BY s.st_id, s.test_grp, course_purchase, ttl_solved, ttl_solved_math
                    )

SELECT ou.test_grp,
       (SUM(ou.money)::float/COUNT(DISTINCT ou.st_id)) AS arpu,
       (SELECT (SUM(inr.money)::float/COUNT(DISTINCT inr.st_id))
          FROM money_data as inr
         WHERE ou.test_grp=inr.test_grp AND inr.ttl_solved>10) AS arpau,
       (COUNT(ou.money)*100.00/COUNT(DISTINCT ou.st_id)) AS cr_percent,
       (SELECT (COUNT(inr.money)*100.00/COUNT(DISTINCT inr.st_id)) 
          FROM money_data as inr
         WHERE ou.test_grp=inr.test_grp AND inr.ttl_solved>10 
        ) AS cr_active_percent,
       (SELECT (SUM(CASE WHEN course_purchase='Math' THEN 1 ELSE 0 END)*100.00/COUNT(DISTINCT inr.st_id))
          FROM money_data as inr
         WHERE ou.test_grp=inr.test_grp AND inr.ttl_solved_math>=2) AS cr_active_math_percent
  FROM money_data as ou
 GROUP BY test_grp; 
 
```

**Итоговый вывод:**

|test_grp |arpu |arpau |cr_percent |cr_active_percent |cr_active_math_percent |
|---:|:---:|:---:|:---:|:---:|:---:|
|control |4,540.98	|10,393.70	|4.92	|11.02	|6.00	|
|pilot |11,508.47	|29,739.58	|10.85	|26.04	|9.09   |